In [ ]:
!pip install networkit


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
import networkit as nk

In [ ]:
!pip install gtfs-kit #install gtfs-kit which will be used to mine the gtfs (transit data)
!pip install osmnx
import pandas as pd
import gtfs_kit as gk
import numpy as np
from scipy.spatial import KDTree
from sklearn.neighbors import BallTree
import numpy as np
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import plotly.express as px
import datetime as dt
from sklearn.metrics.pairwise import haversine_distances
import ast
import re


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
#get nodes and edges
path_edges = "https://github.com/deepikarao281199/CSCI6364_MIND_THE_GAP/raw/refs/heads/main/saopaulo_edges.csv"
edges_df = pd.read_csv(path_edges)#load transit graph - edges
path_nodes = "https://github.com/deepikarao281199/CSCI6364_MIND_THE_GAP/raw/refs/heads/main/saopaulo_nodes.csv"
nodes_df = pd.read_csv(path_nodes)#load transit graph - nodes

In [ ]:
# load the edges and nodes - then create the graph
edges_df['2'] = edges_df['2'].apply(lambda x: ast.literal_eval(re.sub(r"np\.float64\(([-0-9.eE]+)\)", r"\1", x)))#convert 'dict' to just dict - np.float objects caused error so we replace it
nodes_df['1'] = nodes_df['1'].apply(lambda x: ast.literal_eval(re.sub(r"np\.float64\(([-0-9.eE]+)\)", r"\1", x)))#convert 'dict' to just dict
edges_back = list(edges_df.itertuples(index = False, name = None))
nodes_back =  list(nodes_df.itertuples(index = False, name = None))
G = nx.MultiDiGraph()#create empty graph
G.add_nodes_from(nodes_back)
G.add_edges_from(edges_back)
print(G)

MultiDiGraph with 22056 nodes and 102362 edges


In [ ]:
correct_nodes, correct_edges = 22056, 102362
if (len(G.nodes) == correct_nodes) and len(G.edges) == correct_edges:
    print("Correct Graph generated!")
else:
    print("Incorrect graph generated, redownload edges and nodes, try again!")

Correct Graph generated!


In [ ]:
#generate networkit graph
G_nx = G.copy()
mapping = {label: i for i, label in enumerate(G_nx.nodes())}
G_nx = nx.relabel_nodes(G_nx, mapping)

Gtime_nk = nk.graph.Graph(G_nx.number_of_nodes(), weighted=True, directed=G_nx.is_directed())
Gdist_nk = nk.graph.Graph(G_nx.number_of_nodes(), weighted=True, directed=G_nx.is_directed())
for u, v, data in G_nx.edges(data=True):
    time = data.get("time")
    dist = data.get("distance")
    Gtime_nk.addEdge(u, v, time)
    Gdist_nk.addEdge(u, v, dist)

The transit network is build. The next step is to find the average transit time from each neightborhood. Metrics we will get:
- number of walkable stops from each neighbourhood centroid
- num of schools/hospitals/clincs/pharmacies/grocery etc reachable within 30 mins
- mean travel time to key services eg. average time to nearest groceries
- average transfer count to key services
- average distance to key services
- num of connected subgraphs - disconnected areas = accessibility gaps

In [ ]:
#using open street map to get walkable paths
places = ['São Paulo, Brazil']
G_walk = ox.graph_from_place(places, network_type="walk")
G_walk

In [ ]:
#load nearest osmx nearest node to each stop - from ML Final Project GTFS
path_gwalk = "https://github.com/deepikarao281199/CSCI6364_MIND_THE_GAP/raw/refs/heads/main/saopaulo_stops_with_osmx.csv"
stops_gwalk = pd.read_csv(path_gwalk).set_index('stop_id')
stops_gwalk_dict = stops_gwalk['gwalk_nearest'].to_dict()
stops_gwalk.head()

,stop_lat,stop_lon,gwalk_nearest
stop_id,,,
18848,-23.554022,-46.671108,11287097095
18849,-23.546498,-46.691141,165473564
18850,-23.558094,-46.660205,3511236618
18851,-23.635039,-46.641239,2396880345
18852,-23.646033,-46.641028,29777031


In [ ]:
#pre-computing walkable edges
osm_walkable = nx.all_pairs_dijkstra_path_length(G_walk, cutoff = 400, weight = 'length')
osm_walkable_edges = {}

for source, dist_dict in osm_walkable:
    for target, dist in dist_dict.items():
        osm_walkable_edges[(source, target)] = dist

# print(osm_walkable_edges)

In [ ]:
#convert to radians to accomodate distance metrics
stop_coords = np.radians(stops_gwalk[['stop_lat', 'stop_lon']]).values
#Ball tree creation using Haversine distance
stops_tree = BallTree(stop_coords, metric = 'haversine')

#a quarter mile or 400 meters is generally considered walkable
earth_r = 6371000  #earths radius
distance = 400 /earth_r #radius for 400 meters

walk_speed = 1.375 #average meters per second - https://en.wikipedia.org/wiki/Preferred_walking_speed
walk_max = 10 #maximum time to walk by transfer in mins

In [ ]:
def get_closest_stop_nh(row):
    lat = np.radians(row['lat'])
    lon = np.radians(row['lon'])
    nh_gwalk_id = row['gwalk_nearest']
    point = [[lat, lon]] #must be 2d
    nearest = stops_tree.query_radius(point, r = distance) #returns indices of stops within distance from neighbourhood centroids - stop_id is the indicies
    nearest = [stops_gwalk.index[int(stop)] for stop in nearest[0]]
    walkable_stops = []
    for near in nearest:
        stop_gwalk_id = stops_gwalk_dict[near]
        if ((nh_gwalk_id, stop_gwalk_id)  in osm_walkable_edges) or ((stop_gwalk_id, nh_gwalk_id) in osm_walkable_edges): #if the edge is in osm_walkable_edges then its is walkable so we keep it
            walkable_stops.append(near)
    return walkable_stops

def get_closest_stop_service(row):
    lat = np.radians(row['latitude'])
    lon = np.radians(row['longitude'])
    serv_gwalk_id = row['gwalk_nearest']
    point = [[lat, lon]] #must be 2d
    nearest = stops_tree.query_radius(point, r = distance) #returns indices of stops within distance from service coordinates- stop_id is the indicies
    nearest = [stops_gwalk.index[int(stop)] for stop in nearest[0]]
    walkable_stops = []
    for near in nearest:
        stop_gwalk_id = stops_gwalk_dict[near]
        if ((serv_gwalk_id, stop_gwalk_id)  in osm_walkable_edges) or ((stop_gwalk_id, serv_gwalk_id) in osm_walkable_edges): #if the edge is in osm_walkable_edges then its is walkable so we keep it
            walkable_stops.append(near)
    return walkable_stops

In [ ]:
#pre-compute nearest stops from neighbourhood - store in list
tract_centroids_path = "https://github.com/deepikarao281199/CSCI6364_MIND_THE_GAP/raw/refs/heads/main/sao_paulo_city_tracts.csv"
tract_centroids_df = pd.read_csv(tract_centroids_path, encoding='utf-8-sig')
# tract_centroids_df.head(5)
nh_withGWalk = tract_centroids_df.copy()
nh_withGWalk['gwalk_nearest'] = ox.distance.nearest_nodes(G_walk, nh_withGWalk['lon'], nh_withGWalk['lat'])
nh_withGWalk['nearest_stop_ids'] = nh_withGWalk[['lat', 'lon', 'gwalk_nearest']].apply(get_closest_stop_nh, axis = 1)#returns nearest stop_ids for each node
nh_withGWalk['num_walkable_stops'] = nh_withGWalk['nearest_stop_ids'].apply(lambda x: len(x))#number of walkable stops from each neighbourhood centroid


In [ ]:
#pre-compute nearest stops from service(healthcare) - stored in list
hos_path = "https://github.com/deepikarao281199/CSCI6364_MIND_THE_GAP/raw/refs/heads/main/sao_paulo_healthcare_facilities.csv"
hos_df = pd.read_csv(hos_path)
hos_withGWalk = hos_df.copy()
hos_withGWalk['gwalk_nearest'] = ox.distance.nearest_nodes(G_walk, hos_withGWalk['longitude'], hos_withGWalk['latitude'])
hos_withGWalk['nearest_stop_ids'] = hos_withGWalk[['latitude', 'longitude', 'gwalk_nearest']].apply(get_closest_stop_service, axis = 1)#returns nearest stop_id for each node
hos_withGWalk.head()

,osm_id,osm_type,name,amenity,healthcare,operator,operator_type,emergency,beds,addr_street,phone,latitude,longitude,gwalk_nearest,nearest_stop_ids
0,840695877,node,Hospital São Paulo,hospital,hospital,NaN,NaN,NaN,NaN,Rua Napoleão de Barros,+55 11 576-4000,-23.597777,-46.643533,9517237493,"[790015218, 790015103, 790015104, 920015205, 9..."
1,840722121,node,Hospital Amparo Maternal,hospital,hospital,NaN,NaN,NaN,NaN,Rua Loefgren,+55 11 5089-8277,-23.600057,-46.643394,8525056381,"[790015143, 790015100, 790015142, 790015218, 7..."
2,1590255045,node,Hospital Family,hospital,hospital,NaN,NaN,NaN,NaN,Rua João Santucci,NaN,-23.605742,-46.750509,9327403849,"[9607460, 9607104, 960011375, 9607103, 9607453..."
3,1590265394,node,Intermédica Centro Clínico Taboão da Serra,hospital,hospital,NaN,NaN,no,NaN,NaN,NaN,-23.606907,-46.753860,9327403870,"[9607442, 9607437, 170011305, 9607462, 9607460..."
4,1590265395,node,Fisioterapia Ortoclimed,hospital,hospital,NaN,NaN,no,NaN,NaN,NaN,-23.606632,-46.754225,9327403870,"[9607442, 9607437, 170011305, 9607462, 9607460..."


In [ ]:
#pre-compute nearest stops from service(groceries) - stored in list
groc_path = "https://github.com/deepikarao281199/CSCI6364_MIND_THE_GAP/raw/refs/heads/main/sao_paulo_groceries.csv"
groc_df = pd.read_csv(groc_path)
groc_withGWalk = groc_df.copy()
groc_withGWalk['gwalk_nearest'] = ox.distance.nearest_nodes(G_walk, groc_withGWalk['longitude'], groc_withGWalk['latitude'])
groc_withGWalk['nearest_stop_ids'] = groc_withGWalk[['latitude', 'longitude', 'gwalk_nearest']].apply(get_closest_stop_service, axis = 1)#returns nearest stop_id for each node
groc_withGWalk.head()

,osm_id,osm_type,name,shop,operator,brand,addr_street,phone,opening_hours,latitude,longitude,gwalk_nearest,nearest_stop_ids
0,468410592,node,Hipermercado Zaffari,supermarket,NaN,NaN,NaN,NaN,NaN,-23.527030,-46.680669,5067725753,"[610016827, 610016826, 60014604]"
1,491895198,node,Rycoy Supermercado,supermarket,NaN,NaN,NaN,NaN,NaN,-23.652532,-46.733733,8648768302,"[450011864, 450011863, 450011752, 450011751, 4..."
2,552899310,node,Sonda Hipermercado,supermarket,NaN,NaN,NaN,NaN,NaN,-23.525261,-46.546788,627705985,"[6009710, 600012354, 600012355, 600012358, 600..."
3,599006044,node,Carrefour Express,convenience,NaN,Carrefour Express,NaN,NaN,NaN,-23.553053,-46.634936,13030170534,"[800016574, 8010186, 800016572, 800016582, 188..."
4,631974244,node,Mercadinho,supermarket,NaN,NaN,NaN,NaN,NaN,-23.552410,-46.627578,2435474855,"[800015932, 800012681, 4910058, 800012673, 800..."


In [ ]:
#pre-compute nearest stops from service(groceries) - stored in list
school_path = "https://github.com/deepikarao281199/CSCI6364_MIND_THE_GAP/raw/refs/heads/main/sao_paulo_schools_k12.csv"
school_df = pd.read_csv(school_path)
school_withGWalk = school_df.copy()
school_withGWalk['gwalk_nearest'] = ox.distance.nearest_nodes(G_walk, school_withGWalk['longitude'], school_withGWalk['latitude'])
school_withGWalk['nearest_stop_ids'] = school_withGWalk[['latitude', 'longitude', 'gwalk_nearest']].apply(get_closest_stop_service, axis = 1)#returns nearest stop_id for each node
school_withGWalk.head()

,osm_id,osm_type,name,amenity,operator,operator_type,addr_street,phone,website,latitude,longitude,gwalk_nearest,nearest_stop_ids
0,638425993,node,Colégio Presbiteriano do Brás,school,NaN,NaN,NaN,NaN,NaN,-23.541214,-46.599650,12369793145,"[810583, 80014503, 80014523, 80014502, 8001452..."
1,696175114,node,Coreo Dança,school,NaN,NaN,NaN,NaN,NaN,-23.635853,-46.698141,12802948955,"[7206965, 720011669, 720011736, 720011737, 720..."
2,745728710,node,Anglo Vestibulares,school,NaN,NaN,NaN,NaN,NaN,-23.563672,-46.634874,12539602747,"[490011858, 4906338, 490001818, 4910064, 49000..."
3,922094743,node,Lycee Pasteur,school,NaN,NaN,NaN,NaN,NaN,-23.592263,-46.629856,922094747,"[920016189, 9206443, 9206550]"
4,1040038931,node,Fátima,school,NaN,NaN,NaN,NaN,NaN,-23.597766,-46.525435,1039927806,"[7805205, 780006642, 780006643, 780006697, 780..."


In [ ]:
apsp_time = nk.distance.APSP(Gtime_nk)
apsp_time.run()
apsp_dist = nk.distance.APSP(Gdist_nk)  # multithreaded
apsp_dist.run()

In [ ]:
nh_nodes = list(set([mapping[stop] for nearest in nh_withGWalk['nearest_stop_ids'] for stop in nearest]))
school_nodes = [mapping[stop] for nearest in school_withGWalk['nearest_stop_ids'] for stop in nearest]
hos_nodes = [mapping[stop] for nearest in hos_withGWalk['nearest_stop_ids'] for stop in nearest]
groc_nodes = [mapping[stop] for nearest in groc_withGWalk['nearest_stop_ids'] for stop in nearest]

service_nodes = school_nodes + hos_nodes + groc_nodes
service_nodes = list(set(service_nodes))

N_nh = len(nh_nodes)
N_serv = len(service_nodes)
print("Number of Neighbourhood Nodes:", N_nh)
print("Number of Service Nodes:", N_serv)


#created indexing
nh_index = {n: i for i, n in enumerate(nh_nodes)}
srv_index = {s: i for i, s in enumerate(service_nodes)}

#create matrix instead for faster indexing and hopefully less ram
dist_matrix = np.full((N_nh, N_serv), -1, dtype=float)
time_matrix = np.full((N_nh, N_serv), -1, dtype=float)

for n in nh_nodes:
    ni = nh_index[n]
    for s in service_nodes:
        si = srv_index[s]
        time = apsp_time.getDistance(n, s)
        dist = apsp_dist.getDistance(n, s)
        time_matrix[ni, si] = time
        dist_matrix[ni, si] = dist

Number of Neighbourhood Nodes: 21280
Number of Service Nodes: 10574


In [ ]:
hos_dict = dict(zip(hos_withGWalk['osm_id'], hos_withGWalk['nearest_stop_ids']))
groc_dict = dict(zip(groc_withGWalk['osm_id'], groc_withGWalk['nearest_stop_ids']))
school_dict = dict(zip(school_withGWalk['osm_id'], school_withGWalk['nearest_stop_ids']))

In [ ]:
#num of schools/hospitals/clincs/pharmacies/grocery etc reachable within 30 mins
def num_service_reachable(nh_stops, service_type):
    #get the stops within walking distance from neighbourhood
    if service_type == 'hospital':
        service_dict = hos_dict
    elif service_type == 'school':
        service_dict = school_dict
    elif service_type == 'grocery':
        service_dict = groc_dict

    reachable = []
    for nh_stop in nh_stops:
        nh_matid = nh_index[mapping[nh_stop]]
        #for each stop get the stops for each service and then use the pre - computed to get the time
        for i, serv_stops in enumerate(service_dict.values()):
            if len(serv_stops) != 0: #not empty
                for serv_stop in serv_stops:
                    serv_matid = srv_index[mapping[serv_stop]]
                    service_time = time_matrix[(nh_matid, serv_matid)]
                    if service_time != float('inf') and service_time <= 30:#the time is in mins so anything > 30 we don't add to reachable
                        reachable.append(i)

    return len(set(reachable))

In [ ]:
#number of hospitals within 30 mins for each neighbourhood
new_tracts_df = nh_withGWalk.copy()
new_tracts_df['num_hospitals_in_30'] = new_tracts_df['nearest_stop_ids'].apply(lambda x: num_service_reachable(x, "hospital"))
print("hospitals done")
#number of schools within 30 mins for each neighbourhood
new_tracts_df['num_schools_in_30'] = new_tracts_df['nearest_stop_ids'].apply(lambda x: num_service_reachable(x, "school"))
print("schools done")
#number of grocery stores within 30 mins for each neighbourhood
new_tracts_df['num_grocery_in_30'] = new_tracts_df['nearest_stop_ids'].apply(lambda x: num_service_reachable(x, "grocery"))
print("groceries done")
new_tracts_df.head()

hospitals done
schools done
groceries done


,CD_SETOR,NM_MUN,CD_DIST,NM_DIST,lat,lon,gwalk_nearest,nearest_stop_ids,num_walkable_stops,num_hospitals_in_30,num_schools_in_30,num_grocery_in_30
0,355030801000001,São Paulo,355030801,Água Rasa,-23.567836,-46.570841,1420391152,"[10008742, 10008741, 10008760, 10008759, 10008...",6,83,319,502
1,355030801000002,São Paulo,355030801,Água Rasa,-23.566562,-46.568271,688021972,[10008759],1,0,27,24
2,355030801000003,São Paulo,355030801,Água Rasa,-23.568594,-46.567571,2795091218,"[10008758, 10008757]",2,58,260,413
3,355030801000004,São Paulo,355030801,Água Rasa,-23.570255,-46.568904,688021935,"[10008745, 105388, 10008752, 10008753, 1000875...",8,71,307,475
4,355030801000005,São Paulo,355030801,Água Rasa,-23.571521,-46.570585,4569313209,"[10008745, 105388, 10008742, 10008741]",4,71,307,475


In [ ]:
new_tracts_df.to_csv("content/tracts_new1", index = False)

In [ ]:
#mean travel time to key services eg. average time to nearest groceries
def av_service_reachable(nh_stops, service_type):
    if service_type == 'hospital':
        service_dict = hos_dict
    elif service_type == 'school':
        service_dict = school_dict
    elif service_type == 'grocery':
        service_dict = groc_dict

    time_av = []
    for nh_stop in nh_stops:
        nh_matid = nh_index[mapping[nh_stop]]
        time_stop = []
        #for each stop get the stops for each service and then use the pre - computed to get the time
        for i, serv_stops in enumerate(service_dict.values()):
            if len(serv_stops) != 0: #not empty
                for serv_stop in serv_stops:
                    serv_matid = srv_index[mapping[serv_stop]]
                    service_time = time_matrix[(nh_matid, serv_matid)]
                    if service_time != float('inf') and service_time <= 180: #a service more than 3 hours away is not considered
                        time_stop.append(service_time)
        time_av.append(np.mean(time_stop))
    return np.mean(time_av)
    #return average of reachle


In [ ]:
#average time to reach a hospital if its less than 3 hours way for each neighbourhood
new_tracts_df['av_time_hos'] = new_tracts_df['nearest_stop_ids'].apply(lambda x: av_service_reachable(x, "hospital"))
print("hospitals done")
#average time to reach a school if its less than 3 hours way for each neighbourhood
new_tracts_df['av_time_school'] = new_tracts_df['nearest_stop_ids'].apply(lambda x: av_service_reachable(x, "school"))
print("schools done")
#average time to reach a grocery if its less than 3 hours way for each neighbourhood
new_tracts_df['av_time_groc'] = new_tracts_df['nearest_stop_ids'].apply(lambda x: av_service_reachable(x, "grocery"))
print("groceries done")
new_tracts_df.to_csv("content/tracts_new2", index = False)
new_tracts_df.head()

/Users/rajeevsingh/github/personal_projects/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/rajeevsingh/github/personal_projects/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


hospitals done


/Users/rajeevsingh/github/personal_projects/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/rajeevsingh/github/personal_projects/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


schools done


/Users/rajeevsingh/github/personal_projects/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/rajeevsingh/github/personal_projects/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


groceries done


,CD_SETOR,NM_MUN,CD_DIST,NM_DIST,lat,lon,gwalk_nearest,nearest_stop_ids,num_walkable_stops,num_hospitals_in_30,num_schools_in_30,num_grocery_in_30,av_dist_hos,av_dist_school,av_dist_groc,av_time_hos,av_time_school,av_time_groc
0,355030801000001,São Paulo,355030801,Água Rasa,-23.567836,-46.570841,1420391152,"[10008742, 10008741, 10008760, 10008759, 10008...",6,83,319,502,16.474461,18.063175,16.558285,50.536946,56.472793,50.954896
1,355030801000002,São Paulo,355030801,Água Rasa,-23.566562,-46.568271,688021972,[10008759],1,0,27,24,21.204998,22.067965,21.304178,70.490057,74.522046,70.864702
2,355030801000003,São Paulo,355030801,Água Rasa,-23.568594,-46.567571,2795091218,"[10008758, 10008757]",2,58,260,413,18.180310,19.598149,18.275943,56.854349,62.080089,57.300063
3,355030801000004,São Paulo,355030801,Água Rasa,-23.570255,-46.568904,688021935,"[10008745, 105388, 10008752, 10008753, 1000875...",8,71,307,475,17.521852,19.028993,17.614429,53.997717,59.501630,54.444885
4,355030801000005,São Paulo,355030801,Água Rasa,-23.571521,-46.570585,4569313209,"[10008745, 105388, 10008742, 10008741]",4,71,307,475,15.631846,17.308259,15.710389,46.990744,53.279071,47.397438


In [ ]:
def av_service_reachable_dist(nh_stops, service_type):
    if service_type == 'hospital':
        service_dict = hos_dict
    elif service_type == 'school':
        service_dict = school_dict
    elif service_type == 'grocery':
        service_dict = groc_dict

    dist_av = []
    for nh_stop in nh_stops:
        nh_matid = nh_index[mapping[nh_stop]]
        dist_stop = []
        for i, serv_stops in enumerate(service_dict.values()):
            if len(serv_stops) != 0:
                for serv_stop in serv_stops:
                    serv_matid = srv_index[mapping[serv_stop]]
                    service_dist = dist_matrix[(nh_matid, serv_matid)]
                    # Skip if: infinity, -1 (not computed), or > 100km cutoff
                    if service_dist != float('inf') and service_dist != -1 and service_dist > 0 and service_dist <= 100000:  # 100km in meters
                        dist_stop.append(service_dist / 1000)  # Convert to km
        if dist_stop:
            dist_av.append(np.mean(dist_stop))
    return np.mean(dist_av) if dist_av else np.nan

In [ ]:
# Average DISTANCE to hospitals
new_tracts_df['av_dist_hos'] = new_tracts_df['nearest_stop_ids'].apply(lambda x: av_service_reachable_dist(x, "hospital"))
print("hospitals done")
print(f"  av_dist_hos - min: {new_tracts_df['av_dist_hos'].min():.2f}, max: {new_tracts_df['av_dist_hos'].max():.2f}, mean: {new_tracts_df['av_dist_hos'].mean():.2f}")

# Average DISTANCE to schools
new_tracts_df['av_dist_school'] = new_tracts_df['nearest_stop_ids'].apply(lambda x: av_service_reachable_dist(x, "school"))
print("schools done")
print(f"  av_dist_school - min: {new_tracts_df['av_dist_school'].min():.2f}, max: {new_tracts_df['av_dist_school'].max():.2f}, mean: {new_tracts_df['av_dist_school'].mean():.2f}")

# Average DISTANCE to groceries
new_tracts_df['av_dist_groc'] = new_tracts_df['nearest_stop_ids'].apply(lambda x: av_service_reachable_dist(x, "grocery"))
print("groceries done")
print(f"  av_dist_groc - min: {new_tracts_df['av_dist_groc'].min():.2f}, max: {new_tracts_df['av_dist_groc'].max():.2f}, mean: {new_tracts_df['av_dist_groc'].mean():.2f}")

# Show final dataframe
new_tracts_df.head()

hospitals done
  av_dist_hos - min: 7.68, max: 51.31, mean: 20.10
schools done
  av_dist_school - min: 8.66, max: 55.66, mean: 22.59
groceries done
  av_dist_groc - min: 7.45, max: 51.79, mean: 20.18


,CD_SETOR,NM_MUN,CD_DIST,NM_DIST,lat,lon,gwalk_nearest,nearest_stop_ids,num_walkable_stops,num_hospitals_in_30,num_schools_in_30,num_grocery_in_30,av_dist_hos,av_dist_school,av_dist_groc
0,355030801000001,São Paulo,355030801,Água Rasa,-23.567836,-46.570841,1420391152,"[10008742, 10008741, 10008760, 10008759, 10008...",6,83,319,502,16.474461,18.063175,16.558285
1,355030801000002,São Paulo,355030801,Água Rasa,-23.566562,-46.568271,688021972,[10008759],1,0,27,24,21.204998,22.067965,21.304178
2,355030801000003,São Paulo,355030801,Água Rasa,-23.568594,-46.567571,2795091218,"[10008758, 10008757]",2,58,260,413,18.180310,19.598149,18.275943
3,355030801000004,São Paulo,355030801,Água Rasa,-23.570255,-46.568904,688021935,"[10008745, 105388, 10008752, 10008753, 1000875...",8,71,307,475,17.521852,19.028993,17.614429
4,355030801000005,São Paulo,355030801,Água Rasa,-23.571521,-46.570585,4569313209,"[10008745, 105388, 10008742, 10008741]",4,71,307,475,15.631846,17.308259,15.710389


In [ ]:
new_tracts_df['service_density'] = (
    new_tracts_df['num_hospitals_in_30'] +
    new_tracts_df['num_schools_in_30'] +
    new_tracts_df['num_grocery_in_30']
)
print("service_density done")
new_tracts_df.head()

service_density done


,CD_SETOR,NM_MUN,CD_DIST,NM_DIST,lat,lon,gwalk_nearest,nearest_stop_ids,num_walkable_stops,num_hospitals_in_30,num_schools_in_30,num_grocery_in_30,av_dist_hos,av_dist_school,av_dist_groc,av_time_hos,av_time_school,av_time_groc,service_density
0,355030801000001,São Paulo,355030801,Água Rasa,-23.567836,-46.570841,1420391152,"[10008742, 10008741, 10008760, 10008759, 10008...",6,83,319,502,16.474461,18.063175,16.558285,50.536946,56.472793,50.954896,904
1,355030801000002,São Paulo,355030801,Água Rasa,-23.566562,-46.568271,688021972,[10008759],1,0,27,24,21.204998,22.067965,21.304178,70.490057,74.522046,70.864702,51
2,355030801000003,São Paulo,355030801,Água Rasa,-23.568594,-46.567571,2795091218,"[10008758, 10008757]",2,58,260,413,18.180310,19.598149,18.275943,56.854349,62.080089,57.300063,731
3,355030801000004,São Paulo,355030801,Água Rasa,-23.570255,-46.568904,688021935,"[10008745, 105388, 10008752, 10008753, 1000875...",8,71,307,475,17.521852,19.028993,17.614429,53.997717,59.501630,54.444885,853
4,355030801000005,São Paulo,355030801,Água Rasa,-23.571521,-46.570585,4569313209,"[10008745, 105388, 10008742, 10008741]",4,71,307,475,15.631846,17.308259,15.710389,46.990744,53.279071,47.397438,853


If a tract has less than 2 transit stops within walking distance (400m), it's a "transit desert" - an underserved area/indicating poor transit accessibility.

In [ ]:
new_tracts_df['is_transit_desert'] = (new_tracts_df['num_walkable_stops'] < 2).astype(int)
print("is_transit_desert done")
new_tracts_df.head()

is_transit_desert done


,CD_SETOR,NM_MUN,CD_DIST,NM_DIST,lat,lon,gwalk_nearest,nearest_stop_ids,num_walkable_stops,num_hospitals_in_30,num_schools_in_30,num_grocery_in_30,av_dist_hos,av_dist_school,av_dist_groc,av_time_hos,av_time_school,av_time_groc,service_density,is_transit_desert
0,355030801000001,São Paulo,355030801,Água Rasa,-23.567836,-46.570841,1420391152,"[10008742, 10008741, 10008760, 10008759, 10008...",6,83,319,502,16.474461,18.063175,16.558285,50.536946,56.472793,50.954896,904,0
1,355030801000002,São Paulo,355030801,Água Rasa,-23.566562,-46.568271,688021972,[10008759],1,0,27,24,21.204998,22.067965,21.304178,70.490057,74.522046,70.864702,51,1
2,355030801000003,São Paulo,355030801,Água Rasa,-23.568594,-46.567571,2795091218,"[10008758, 10008757]",2,58,260,413,18.180310,19.598149,18.275943,56.854349,62.080089,57.300063,731,0
3,355030801000004,São Paulo,355030801,Água Rasa,-23.570255,-46.568904,688021935,"[10008745, 105388, 10008752, 10008753, 1000875...",8,71,307,475,17.521852,19.028993,17.614429,53.997717,59.501630,54.444885,853,0
4,355030801000005,São Paulo,355030801,Água Rasa,-23.571521,-46.570585,4569313209,"[10008745, 105388, 10008742, 10008741]",4,71,307,475,15.631846,17.308259,15.710389,46.990744,53.279071,47.397438,853,0


0 = Tract is connected to main transit network
1 = Tract's transit stops are disconnected from main network

In [ ]:
components = list(nx.weakly_connected_components(G))
main_component = max(components, key=len)
isolated_nodes = set()
for comp in components:
    if comp != main_component:
        isolated_nodes.update(comp)

def check_isolated(stop_ids):
    for stop in stop_ids:
        if stop in isolated_nodes:
            return 1
    return 0

new_tracts_df['is_isolated'] = new_tracts_df['nearest_stop_ids'].apply(check_isolated)
print("is_isolated done")
new_tracts_df.head()

is_isolated done


,CD_SETOR,NM_MUN,CD_DIST,NM_DIST,lat,lon,gwalk_nearest,nearest_stop_ids,num_walkable_stops,num_hospitals_in_30,...,num_grocery_in_30,av_dist_hos,av_dist_school,av_dist_groc,av_time_hos,av_time_school,av_time_groc,service_density,is_transit_desert,is_isolated
0,355030801000001,São Paulo,355030801,Água Rasa,-23.567836,-46.570841,1420391152,"[10008742, 10008741, 10008760, 10008759, 10008...",6,83,...,502,16.474461,18.063175,16.558285,50.536946,56.472793,50.954896,904,0,0
1,355030801000002,São Paulo,355030801,Água Rasa,-23.566562,-46.568271,688021972,[10008759],1,0,...,24,21.204998,22.067965,21.304178,70.490057,74.522046,70.864702,51,1,0
2,355030801000003,São Paulo,355030801,Água Rasa,-23.568594,-46.567571,2795091218,"[10008758, 10008757]",2,58,...,413,18.180310,19.598149,18.275943,56.854349,62.080089,57.300063,731,0,0
3,355030801000004,São Paulo,355030801,Água Rasa,-23.570255,-46.568904,688021935,"[10008745, 105388, 10008752, 10008753, 1000875...",8,71,...,475,17.521852,19.028993,17.614429,53.997717,59.501630,54.444885,853,0,0
4,355030801000005,São Paulo,355030801,Água Rasa,-23.571521,-46.570585,4569313209,"[10008745, 105388, 10008742, 10008741]",4,71,...,475,15.631846,17.308259,15.710389,46.990744,53.279071,47.397438,853,0,0


In [ ]:
new_tracts_df.to_csv("content/sao_paulo_tracts_features.csv", index=False)
print("Saved! Shape:", new_tracts_df.shape)
print("Columns:", new_tracts_df.columns.tolist())

Saved! Shape: (27301, 21)
Columns: ['CD_SETOR', 'NM_MUN', 'CD_DIST', 'NM_DIST', 'lat', 'lon', 'gwalk_nearest', 'nearest_stop_ids', 'num_walkable_stops', 'num_hospitals_in_30', 'num_schools_in_30', 'num_grocery_in_30', 'av_dist_hos', 'av_dist_school', 'av_dist_groc', 'av_time_hos', 'av_time_school', 'av_time_groc', 'service_density', 'is_transit_desert', 'is_isolated']
